<h1>はじめに</h1>

番外編2では、loto6のデータをPandasを使ってより分析に適した形状に整理した手順をご紹介します。<br>
手順は以下になります。
1. 当選数字のベクトル表現
2. キャリーオーバー額の1行ずらし
3. 販売実績額の追加<br>

<h1>当選数字のベクトル表現</h1>

まずは必要なライブラリをインポートします。

In [18]:
import pandas as pd

データを読み込みます。

In [19]:
df = pd.read_csv('loto6.csv', encoding="cp932")
#データ確認
df.head()

,抽せん回,抽せん日,曜日,本数字1,本数字2,本数字3,本数字4,本数字5,本数字6,ボーナス数字,...,1等賞金,2等口数,2等賞金,3等口数,3等賞金,4等口数,4等賞金,5等口数,5等賞金,キャリーオーバー
0,1,2000/10/5,木,2,8,10,13,27,30,39,...,45513600,2,40961900,262,375200,12413,6900,174452,1000,0
1,2,2000/10/12,木,1,9,16,20,21,43,5,...,0,7,13188800,195,568100,8426,11500,144860,1000,102580607
2,3,2000/10/19,木,1,5,15,31,36,38,13,...,121348100,8,15762800,164,922700,9450,14100,181437,1000,0
3,4,2000/10/26,木,16,18,26,27,34,40,13,...,0,3,38645500,134,1038200,8675,14100,160892,1000,128819603
4,5,2000/11/2,木,9,15,21,23,27,28,43,...,0,9,13516100,258,565700,16473,7800,239987,1000,263982587


入手できるデータのフォーマットでは、「本数字1〜ボーナス数字」、という項目名で数字が入っています。このままだと、「1が出た時はどうなっていたか？」等の分析がしにくいため、ベクトル表現版を作成します。ベクトル表現の意味が分からない方は、具体的に以下を見ていただけば意味が分かると思います。まずは入れ物のデータフレームを作ります。

In [20]:
adf = pd.DataFrame(index=df.index, columns=range(1, 44), data=0)
#データ確認
adf.head()

,1,2,3,4,5,6,7,8,9,10,...,34,35,36,37,38,39,40,41,42,43
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


ベクトル表現とは、要するに、ロト6の出目である1〜43の項目をつくり、出ていれば1、出ていなければ0とする、ということです。for文を使って値を埋めていきましょう。ここではもう少し工夫して、「本数字1ならば1、2ならば2・・・ボーナスならば7を入れる」という作業を行います。

In [21]:
#1行ごとに実施
for i in range(len(df.index)):

    #本数字1〜ボーナス数字」の部分だけを取り出し
    tdf = df.iloc[i, 3:10]

    #tdfの1要素ごとに実施
    for j in range(len(tdf.index)):

        #adfの該当の場所に数値（本数字1ならば1、2ならば2・・・ボーナスならば7を入れる）
        adf.loc[df.index[i], tdf.iloc[j]] = j+1
        
#データ確認
adf.head()

,1,2,3,4,5,6,7,8,9,10,...,34,35,36,37,38,39,40,41,42,43
0,0,1,0,0,0,0,0,2,0,3,...,0,0,0,0,0,7,0,0,0,0
1,1,0,0,0,7,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,6
2,1,0,0,0,2,0,0,0,0,0,...,0,0,5,0,6,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,5,0,0,0,0,0,6,0,0,0
4,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,7


dfと比較してみると、第1回は2が本数字1なので1、39がボーナス数字なので7、等々、正しく入っていることが確認できました。

<h1>キャリーオーバー額の1行ずらし</h1>

dfを見ると分かるように、現状のデータでは「次回のキャリーオーバー額」が入っています。ただ、実際に購入口数や当選数と影響があるのは「今回のキャリーオーバー額」ですよね。そこで1行ずらしてこのデータを作りましょう。

In [22]:
df['キャリーオーバー_今回'] = df['キャリーオーバー'].shift(1).fillna(0).astype(int)
#データ確認
df.iloc[:, 10:].head(10)

,1等口数,1等賞金,2等口数,2等賞金,3等口数,3等賞金,4等口数,4等賞金,5等口数,5等賞金,キャリーオーバー,キャリーオーバー_今回
0,2,45513600,2,40961900,262,375200,12413,6900,174452,1000,0,0
1,0,0,7,13188800,195,568100,8426,11500,144860,1000,102580607,0
2,2,121348100,8,15762800,164,922700,9450,14100,181437,1000,0,102580607
3,0,0,3,38645500,134,1038200,8675,14100,160892,1000,128819603,0
4,0,0,9,13516100,258,565700,16473,7800,239987,1000,263982587,128819603
5,1,400000000,5,46923800,787,357700,36421,6800,512712,1000,124672751,263982587
6,1,348644800,12,16797700,486,497700,23473,9000,354810,1000,0,124672751
7,0,0,2,93073600,183,1220600,10337,19000,216684,1000,206832135,0
8,1,400000000,14,16051500,548,492000,25632,9200,388610,1000,56524796,206832135
9,0,0,11,19637700,341,760100,17984,12700,311772,1000,296543942,56524796


1行ずれたギャリーオーバー額が出来ています。

<h1>販売実績額の追加</h1>

上記のデータに、さらにスクレイピングで取得した販売実績額を加えましょう。

In [23]:
sdf = pd.read_csv("salesData.csv", usecols=['販売実績額'])
#データ確認
sdf.head()

,販売実績額
0,1183183200
1,1218391000
2,1627702400
3,1483327400
4,1714530600


ここまで作ったすべてのデータを結合します。

In [24]:
df = pd.concat([df, sdf, adf], axis=1)
#データ確認
df.head()

,抽せん回,抽せん日,曜日,本数字1,本数字2,本数字3,本数字4,本数字5,本数字6,ボーナス数字,...,34,35,36,37,38,39,40,41,42,43
0,1,2000/10/5,木,2,8,10,13,27,30,39,...,0,0,0,0,0,7,0,0,0,0
1,2,2000/10/12,木,1,9,16,20,21,43,5,...,0,0,0,0,0,0,0,0,0,6
2,3,2000/10/19,木,1,5,15,31,36,38,13,...,0,0,5,0,6,0,0,0,0,0
3,4,2000/10/26,木,16,18,26,27,34,40,13,...,5,0,0,0,0,0,6,0,0,0
4,5,2000/11/2,木,9,15,21,23,27,28,43,...,0,0,0,0,0,0,0,0,0,7


In [25]:
#項目名の確認
df.columns

Index([       '抽せん回',        '抽せん日',          '曜日',        '本数字1',
              '本数字2',        '本数字3',        '本数字4',        '本数字5',
              '本数字6',      'ボーナス数字',        '1等口数',        '1等賞金',
              '2等口数',        '2等賞金',        '3等口数',        '3等賞金',
              '4等口数',        '4等賞金',        '5等口数',        '5等賞金',
          'キャリーオーバー', 'キャリーオーバー_今回',       '販売実績額',             1,
                   2,             3,             4,             5,
                   6,             7,             8,             9,
                  10,            11,            12,            13,
                  14,            15,            16,            17,
                  18,            19,            20,            21,
                  22,            23,            24,            25,
                  26,            27,            28,            29,
                  30,            31,            32,            33,
                  34,            35,            36,           

最後に保存して終了です。

In [26]:
df.to_csv("loto6_allData.csv")

やはりPandasは便利ですね。